In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip uninstall tensorflow
!pip install tensorflow==2.2
!pip uninstall keras
!pip install keras==2.1.2
!pip uninstall numpy
!pip install numpy==1.19

Found existing installation: tensorflow 2.9.1
Uninstalling tensorflow-2.9.1:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.7/dist-packages/tensorflow-2.9.1.dist-info/*
    /usr/local/lib/python3.7/dist-packages/tensorflow/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-2.9.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 516.2 MB 4.4 kB/s 
     |████████████████████████████████| 454 kB 67.1 MB/s 
     |████████████████████████████████| 2.9 MB 56.8 MB/s 
     |████████████████████████████████| 3.0 MB 52.2 MB/s 
     |████████████████████████████████| 26.1 MB 1.3 MB/s 
  Attempting uninstall: tensorflow-estimator
  

Found existing installation: keras 2.9.0
Uninstalling keras-2.9.0:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/keras-2.9.0.dist-info/*
    /usr/local/lib/python3.7/dist-packages/keras/*
Proceed (y/n)? y
  Successfully uninstalled keras-2.9.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 304 kB 27.2 MB/s 


Found existing installation: numpy 1.21.6
Uninstalling numpy-1.21.6:
  Would remove:
    /usr/bin/f2py
    /usr/local/bin/f2py
    /usr/local/bin/f2py3
    /usr/local/bin/f2py3.7
    /usr/local/lib/python3.7/dist-packages/numpy-1.21.6.dist-info/*
    /usr/local/lib/python3.7/dist-packages/numpy.libs/libgfortran-2e0d59d6.so.5.0.0
    /usr/local/lib/python3.7/dist-packages/numpy.libs/libopenblasp-r0-2d23e62b.3.17.so
    /usr/local/lib/python3.7/dist-packages/numpy.libs/libquadmath-2d0c479f.so.0.0.0
    /usr/local/lib/python3.7/dist-packages/numpy/*
Proceed (y/n)? y
  Successfully uninstalled numpy-1.21.6
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 14.6 MB 30.6 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xarray-einstats 0.2.2 requires numpy>=1.21, but you have 

In [ ]:
import os
os.kill(os.getpid(), 9)

In [ ]:
import tensorflow as tf

print(tf.keras.__version__)

2.3.0-tf


In [ ]:
import os
import re
import csv
import codecs
import numpy as np
import pandas as pd

from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from string import punctuation
from keras.preprocessing.text import Tokenizer
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation

import tensorflow as tf
from sklearn.model_selection import train_test_split
import unicodedata
import io
import time

In [ ]:
path_to_file = "/content/drive/MyDrive/Dataset_Anglais/Final_Train_data.txt"
# Preview the dataset
lines = io.open(path_to_file, encoding='utf-8').read().strip().split('\n')

[[w for w in l.split('\t')[:2]]  for l in lines[:5]]

[['i cannot see why not if the panel has usb capabilities',
  'can it be charged with a solar panel ?'],
 ['usually about two years from purchase but three months from time of opening',
  'what is the expired date for the product ?'],
 ['pretty darn well i used it in my apartment that is almost entirely carpeted and it worked well',
  'how well does it roll on carpet ?'],
 ['i have looked at several other sites that sell the mp530 and all say it has a one year warranty with fast replace or something i wish it was two years',
  'warranty which is it ?'],
 ['this is omtp based on the few i have received from them the mh1 is omtp the mh1c is ctia this is the mh1',
  'is this the omtp version or the ctia ?']]

In [ ]:
word_pairs = [[w for w in l.split('\t')[:2]]  for l in lines]

In [ ]:
texts_1 = [word_pairs[i][0] for i in range(len(word_pairs))]
texts_2 = [word_pairs[i][1] for i in range(len(word_pairs))]

In [ ]:
#PreTreatment

def preTreatement(w):
  
  w = w.lower().strip()

  # unicode to ascii
  w = ''.join(c for c in unicodedata.normalize('NFD', w) if unicodedata.category(c) != 'Mn')
  #delete extra white space at the end & begining
  w = w.strip()

  # add the start and end tokens
  w = '<start> ' + w + ' <end>'
    
  return w

In [ ]:
# making the pre treatement to the sentences, and return the dataset in the format : answers, questions
def extractSentences():

  answer = [preTreatement(w) for w in texts_1]
  question = [preTreatement(w) for w in texts_2]


  return answer, question

In [ ]:
#Tokenization

def tokenize(lang):
  lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
  lang_tokenizer.fit_on_texts(lang)

  tensor = lang_tokenizer.texts_to_sequences(lang)
  tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,padding='post')

  return tensor, lang_tokenizer

In [ ]:
# Load the dataset + Tokenization for the inputs and outputs
#input= Answer
#target= Question
def loadSentences():
    
  # creating cleaned input, output pairs
  inputLang, targetLang = extractSentences()

  inputTensor, inputLang_tokenizer = tokenize(inputLang)
  targetTensor, targetLang_tokenizer = tokenize(targetLang)

  return inputTensor, targetTensor, inputLang_tokenizer, targetLang_tokenizer

In [ ]:
# Load input and output tensors

inputTensor, targetTensor, inputLang, targetLang = loadSentences()

# Calculate max_length of the target tensors
maxLengthTarget, maxLengthInput = targetTensor.shape[1], inputTensor.shape[1]

In [ ]:
# Creating training and validation sets using an 80-20 split (you must also leave another 20% aside for the test ... we have left this 20% in a separate file)

#inputTensor_train, inputTensor_val, targetTensor_train, targetTensor_val = train_test_split(inputTensor, targetTensor, test_size=0.2)
inputTensor_train = inputTensor
targetTensor_train = targetTensor
# Show length
print(len(inputTensor_train), len(targetTensor_train))

180000 180000


In [ ]:
#Convert from index to word

def convert(lang, tensor):
  for t in tensor:
    if t!=0:
      print ("%d ----> %s" % (t, lang.index_word[t]))

In [ ]:
#Definition of hyper Parameters

bufferSize = len(inputTensor_train)
batchSize = 64
stepsPerEpoch = len(inputTensor_train)//batchSize
embeddingDimension = 256
units = 1024
vocabInputSize = len(inputLang.word_index)+1
vocabTargetSize = len(targetLang.word_index)+1

dataset = tf.data.Dataset.from_tensor_slices((inputTensor_train, targetTensor_train)).shuffle(bufferSize)
dataset = dataset.batch(batchSize, drop_remainder=True)

#example of input and target batches

exampleInputBatch, exampleTargetBatch = next(iter(dataset))
exampleInputBatch.shape, exampleTargetBatch.shape

(TensorShape([64, 42]), TensorShape([64, 42]))

In [ ]:
# Encoder class

class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embeddingDimension, enc_units, batch_sz):
    super(Encoder, self).__init__()
    self.batch_sz = batch_sz
    self.enc_units = enc_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embeddingDimension)
    self.gru = tf.keras.layers.GRU(self.enc_units, return_sequences=True, return_state=True, recurrent_initializer='glorot_uniform')

  def call(self, x, hidden):
    x = self.embedding(x)
    output, state = self.gru(x, initial_state = hidden)
    return output, state

  def initialize_hidden_state(self):
    return tf.zeros((self.batch_sz, self.enc_units))

In [ ]:
encoder = Encoder(vocabInputSize, embeddingDimension, units, batchSize)

sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_hidden = encoder(exampleInputBatch, sample_hidden)
print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))

Encoder output shape: (batch size, sequence length, units) (64, 42, 1024)
Encoder Hidden state shape: (batch size, units) (64, 1024)


In [ ]:
#Attention mechanism class

class BahdanauAttention(tf.keras.layers.Layer):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)

  def call(self, query, values):
    
    query_with_time_axis = tf.expand_dims(query, 1)
    score = self.V(tf.nn.tanh(self.W1(query_with_time_axis) + self.W2(values)))

    attention_weights = tf.nn.softmax(score, axis=1)

    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

In [ ]:
attention_layer = BahdanauAttention(10)
attention_result, attention_weights = attention_layer(sample_hidden, sample_output)

print("Attention result shape: (batch size, units) {}".format(attention_result.shape))
print("Attention weights shape: (batchSize, sequence_length, 1) {}".format(attention_weights.shape))

Attention result shape: (batch size, units) (64, 1024)
Attention weights shape: (batchSize, sequence_length, 1) (64, 42, 1)


In [ ]:
#Decoder class

class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embeddingDimension, dec_units, batch_sz):
    super(Decoder, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embeddingDimension)
    self.gru = tf.keras.layers.GRU(self.dec_units, return_sequences=True, return_state=True, recurrent_initializer='glorot_uniform')
    self.fc = tf.keras.layers.Dense(vocab_size)
    

    self.attention = BahdanauAttention(self.dec_units)

  def call(self, x, hidden, enc_output):
    
    context_vector, attention_weights = self.attention(hidden, enc_output)

    x = self.embedding(x)
    
    x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

    output, state = self.gru(x)

    output = tf.reshape(output, (-1, output.shape[2]))
    
    x = self.fc(output)

    return x, state, attention_weights

In [ ]:
decoder = Decoder(vocabTargetSize, embeddingDimension, units, batchSize)

sample_decoder_output, _, _ = decoder(tf.random.uniform((batchSize, 1)),
                                      sample_hidden, sample_output)

print ('Decoder output shape: (batchSize, vocab size) {}'.format(sample_decoder_output.shape))

Decoder output shape: (batchSize, vocab size) (64, 48839)


In [ ]:
#Define the optimizer and the loss function

optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

In [ ]:
#Checkpoints 

checkpoint_dir = '/content/drive/MyDrive/train_file/52'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer, encoder=encoder, decoder=decoder)
checkpoint_manager = tf.train.CheckpointManager(checkpoint, checkpoint_dir, max_to_keep=2)

In [ ]:
# Passing to the inference

# restore the latest checkpoint 
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [ ]:
#Generate Questions

def evaluate(sentence):

  sentence = preTreatement(sentence)

  inputs = [inputLang.word_index[i] for i in sentence.split(' ') if i in inputLang.word_index.keys()]
  inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs], maxlen=maxLengthInput, padding='post')
  inputs = tf.convert_to_tensor(inputs)

  result = ''

  hidden = [tf.zeros((1, units))]
  enc_out, enc_hidden = encoder(inputs, hidden)

  dec_hidden = enc_hidden
  dec_input = tf.expand_dims([targetLang.word_index['<start>']], 0)

  for t in range(maxLengthTarget):
        
    predictions, dec_hidden, attention_weights = decoder(dec_input, dec_hidden, enc_out)

    predicted_id = tf.argmax(predictions[0]).numpy()

    result += targetLang.index_word[predicted_id] + ' '

    if targetLang.index_word[predicted_id] == '<end>':
      return result, sentence

    # the predicted ID is fed back into the model
    dec_input = tf.expand_dims([predicted_id], 0)
    
  return result, sentence

In [ ]:
def generate_question(sentence):
    
  result, sentence = evaluate(sentence)
  f_res = result[:-6].rstrip()
  return f_res

In [ ]:
# path to the dataset
df= pd.read_pickle("/content/drive/MyDrive/Dataset_Anglais/Final_Test_data.pkl")
df.set_axis(["Answer", "Question"],axis=1,inplace=True)



In [ ]:
len(df)

45000

In [ ]:
#Execute only once to create separate refrences and candidatess files
'''
for i in range(len(df)):
    
    reference_file=open("/content/drive/MyDrive/Refrences.txt", "a+")
    refrence=df['Question'][i]
    reference_file.write(str(refrence)+'\n')
    reference_file.close()
    
    candidate_file=open("/content/drive/MyDrive/Candidate.txt", "a+")
    candidate_file.write(generate_question(df['Answer'][i])+'\n')
    candidate_file.close()
    print(i)
'''


In [ ]:
candidte_file= open("/content/drive/MyDrive/Candidate.txt", "r")
for line in candidte_file:
    
    all_candidates = [line.rstrip('\n') for line in candidte_file]
    print(all_candidates)

['are these covers remoneable for cleaning household use from water ?', 'can i use this product with a exterior door ?', 'do the wheels lock when ready to keep the wheels to the wheels', 'is the clip on wet when you put the phone off the case ?', 'can you use it for iphone 5 ?', 'what size would i need to buy ?', 'what standard light bulb does this fixture have', 'others you send 2 when i check and they are 9 99 this item says the shipping time be shipped from', 'what are the packaging ? i am looking for the white saucer ice packs ?', 'since these are not even on amazon ?', 'i am thinking of making sure the amount of bread less than the cheese does the handles handle potatoes dry ?', 'is this better support with the toothbrush ?', 'can they be used with ikea these bulbs ?', 'is this product available in 120v ?', 'does the charger work in asia where its worldwide voltage but im from venezuela will it work ?', 'will this rack work for the wide roll ?', 'does it come with pre paid', 'if i

In [ ]:
reference_file= open("/content/drive/MyDrive/Refrences.txt", "r")
for line in reference_file:
    all_refrences = [line.rstrip('\n') for line in reference_file]
    print(all_refrences)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
import nltk
nltk.download("punkt")
liste_candidates=[]
for words in all_candidates:
    liste_candidates.append(nltk.word_tokenize(words))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
import nltk
nltk.download("punkt")
liste_ref=[]
for words in all_refrences:
    liste_ref.append(nltk.word_tokenize(words))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
import nltk
nltk.download("punkt")
liste_refrences=[]
liste_of_liste_refrences=[]
for words in all_refrences:
    liste_refrences.append(nltk.word_tokenize(words))
    liste_of_liste_refrences.append(liste_refrences)
    liste_refrences=[]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


**BLEU SCORE**

In [ ]:
#Bleu-1
from nltk.translate.bleu_score import corpus_bleu
from nltk.translate.bleu_score import SmoothingFunction
chencherry = SmoothingFunction()
Bleu_score1= corpus_bleu(liste_of_liste_refrences,liste_candidates,smoothing_function= chencherry.method5, weights=(1,0,0,0))
print(Bleu_score1)

0.42605107883963794


In [ ]:
#Bleu-2
from nltk.translate.bleu_score import corpus_bleu
from nltk.translate.bleu_score import SmoothingFunction
chencherry = SmoothingFunction()
Bleu_score2= corpus_bleu(liste_of_liste_refrences,liste_candidates,smoothing_function= chencherry.method5, weights=(0.5, 0.5,))
print(Bleu_score2)

0.25601986166166024


In [ ]:
#Bleu-3
from nltk.translate.bleu_score import corpus_bleu
from nltk.translate.bleu_score import SmoothingFunction
chencherry = SmoothingFunction()
Bleu_score3= corpus_bleu(liste_of_liste_refrences,liste_candidates,smoothing_function= chencherry.method5, weights=(0.333, 0.333, 0.334,))
print(Bleu_score3)

0.15584763286503206


In [ ]:
#Bleu-4
from nltk.translate.bleu_score import corpus_bleu
from nltk.translate.bleu_score import SmoothingFunction
chencherry = SmoothingFunction()
Bleu_score = corpus_bleu(liste_of_liste_refrences,liste_candidates,smoothing_function= chencherry.method5)
print(Bleu_score)

0.09488653462949177


In [ ]:
#AVG SENTENCE SCORE (Testing)
#Bleu-1
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction
chencherry = SmoothingFunction()
Bleu1= 0
sentence_score =0
for i in range(len(liste_of_liste_refrences)):
  #calcul de score entre chaque pare de question
  sentence_score = sentence_bleu(liste_of_liste_refrences[i],liste_candidates[i],weights=(1,0,0,0),smoothing_function= chencherry.method5)
  #print(sentence_score)
  Bleu1= Bleu1 + sentence_score
  sentence_score =0
  
print(Bleu1/len(liste_candidates))

0.33070539615530115


**METEOR SCORE**

In [ ]:
!pip install git+https://github.com/huggingface/evaluate@a45df1eb9996eec64ec3282ebe554061cb366388



In [ ]:
#Méthode 1
#FROM :https://huggingface.co/spaces/evaluate-metric/meteor
import evaluate
meteor = evaluate.load('meteor')
predictions = all_candidates
references = all_refrences
results = meteor.compute(predictions=predictions, references=references)
print(results)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


{'meteor': 0.15634520743249625}


In [ ]:
#Méthode 2 (Ma fonction)
from nltk.translate.meteor_score import meteor_score


import numpy as np

def corpus_meteor(expected, predicted):
    meteor_score_sentences_list = list()
    #Pour calculer le score entre chaque paire
    [meteor_score_sentences_list.append(meteor_score(expect, predict)) for expect, predict in zip(expected, predicted)]
    #la moyenne de la somme des scores de chaque paire
    meteor_score_res = np.mean(meteor_score_sentences_list)
    return meteor_score_res

Score_meteor= corpus_meteor(liste_of_liste_refrences,liste_candidates)
print(Score_meteor)

0.15634520743249625


**Rouge Score**

In [ ]:
#https://github.com/pltrdy/rouge
!pip install rouge

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#Méthode 1
from rouge import FilesRouge

files_rouge = FilesRouge()
hyp_path='/content/drive/MyDrive/Candidate.txt'
ref_path='/content/drive/MyDrive/Refrences.txt'
#score total: la moyenne des scores entre chaque paire
scores = files_rouge.get_scores(hyp_path, ref_path, avg=True)
print(scores)

{'rouge-1': {'r': 0.23960219858062984, 'p': 0.2961205133487821, 'f': 0.24420215181140306}, 'rouge-2': {'r': 0.05191019823902833, 'p': 0.06165571753228472, 'f': 0.051120672211786476}, 'rouge-l': {'r': 0.2237227355721377, 'p': 0.27370419613807073, 'f': 0.22720343317384112}}


In [ ]:
#Méthode2
#https://gist.github.com/jamescalam/eca142d1dd7eda4d1dea5182541077cc#file-rouge_avg-ipynb
from rouge import Rouge
model_out = all_candidates
reference = all_refrences
rouge = Rouge()
rouge.get_scores(model_out, reference, avg=True)

{'rouge-1': {'r': 0.2396011738448105,
  'p': 0.29612215556952315,
  'f': 0.2442020229675784},
 'rouge-2': {'r': 0.05191135182462443,
  'p': 0.061657087689788934,
  'f': 0.05112180825196985},
 'rouge-l': {'r': 0.22372135795152073,
  'p': 0.2737053402073593,
  'f': 0.22720292657231878}}

In [ ]:
#https://github.com/li-plus/rouge-metric
!pip install rouge-metric

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 151 kB 25.6 MB/s 


In [ ]:
#creation de liste de lite non tokenisé pour la ref
liste_ref=[]
liste_of_liste_ref=[]
for i in all_refrences:
    liste_ref.append(i)
    liste_of_liste_ref.append(liste_ref)
    liste_ref=[]

In [ ]:
#Méthode3

#https://github.com/li-plus/rouge-metric
from rouge_metric import PyRouge


hypotheses = all_candidates #[1,2]
references = liste_of_liste_ref#[[1],[2]]

# Evaluate document-wise ROUGE scores
'''rouge = PyRouge(rouge_n=(1, 2, 4), rouge_l=True, rouge_w=False,
                rouge_w_weight=1.2, rouge_s=False, rouge_su=False, skip_gap=4)'''

rouge = PyRouge(rouge_l=True)
scores = rouge.evaluate(hypotheses, references)
print(scores)

{'rouge-1': {'r': 0.23249238598163474, 'p': 0.2814380111449128, 'f': 0.25463446055281685}, 'rouge-2': {'r': 0.05168475552122109, 'p': 0.05982955444179317, 'f': 0.05545971445805076}, 'rouge-l': {'r': 0.21695355562055238, 'p': 0.2599489854965942, 'f': 0.23651313138878055}}
